In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'/home/ecotner/neural_tsp'

In [215]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from neural_tsp.oneshot.train import train
from neural_tsp.libs.environment import TSPEnvironment
from neural_tsp.oneshot.model import PolicyNetwork, QNetwork

In [281]:
p = PolicyNetwork()
q = QNetwork()

p.load_state_dict(torch.load("actor_model.torch"))
q.load_state_dict(torch.load("critic_model.torch"))

<All keys matched successfully>

In [304]:
train(200_000, 3e-10)

Episode 100/200000; loss: 29.038278579711914; Q-value: -24.52947998046875; reward: -19.140762329101562
Episode 200/200000; loss: 143.42735290527344; Q-value: -24.956605911254883; reward: -12.980489730834961
Episode 300/200000; loss: 130.85411071777344; Q-value: -35.50004577636719; reward: -24.060897827148438
Episode 400/200000; loss: 3.6926651000976562; Q-value: -26.667251586914062; reward: -24.745620727539062
Episode 500/200000; loss: 649.5120849609375; Q-value: -51.89666748046875; reward: -26.41114044189453
Episode 600/200000; loss: 14.718133926391602; Q-value: -22.231067657470703; reward: -18.39464569091797
Episode 700/200000; loss: 101.63416290283203; Q-value: -24.49675750732422; reward: -14.415380477905273
Episode 800/200000; loss: 266.55035400390625; Q-value: -46.087806701660156; reward: -62.41417694091797
Episode 900/200000; loss: 2301.056884765625; Q-value: -22.74789047241211; reward: -70.71722412109375
Episode 1000/200000; loss: 161.74758911132812; Q-value: -23.835308074951172

In [ ]:
train_df = pd.read_csv("train.log")
train_df.head()

In [ ]:
ma = train_df.ewm(alpha=1e-2).mean()

In [ ]:
fig, axes = plt.subplots(nrows=3)

for i, label in enumerate(ma.columns):
    axes[i].plot(ma[label], label=label, color=f"C{i}")

plt.subplots_adjust(right=2, top=2)
plt.show()